#  Design Patterns Detection

## importing libraries 

In [108]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from matplotlib import pyplot as plt
from sklearn.cluster import DBSCAN,KMeans
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import coo_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import os
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import  cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


## creating a list for each type

In [73]:
creational=['Abstract_factory','Builder','Factory','Prototype','Singleton']
structural=['Adapter','Bridge','Composite','Decorator','Facade','Flyweight','Proxy']
behavioral=['Chain_of_responsibility','Command','Interpreter','Iterator','Mediator','Memento','Observer','State','Strategy','Template','Visitor']
df_dir = 'Dp_trainset'
test_df_dir ='Dp_testset'

## imput function

In [74]:
def imput(df):
    for c in df:
        df[c].fillna((df[c].mean()), inplace=True)
    return df


## preprocessing function that reads class file and do input for it after skipping the first three columns, because the most useful one is class file and represent it as a row in our data set by do sum on axis=0

In [75]:
def prepro(df_dir):
    X,y=[],[]
    empty=0
    for foldername in os.listdir(df_dir):
        lable='n'
        if foldername in creational:
            lable=0
        elif foldername in structural:
            lable=1
        elif foldername in behavioral:
            lable=2
        for folder in os.listdir(df_dir+'/'+foldername):
            row=[]
            df_class = pd.read_csv(df_dir+'/'+foldername+'/'+folder+'/ckmetrics/class.csv')
            if len(df_class)==0:
                empty+=1
                continue
            class_count=(df_class.type=='class').sum()
            inter_count=(df_class.type=='interface').sum()
            df_class=df_class.iloc[:,3:]
            df_class=imput(df_class)
            row=np.append(row,df_class.sum(axis=0))
            row=np.append(row,class_count)
            row=np.append(row,inter_count)
            X.append(row)
            y.append(lable)
    return pd.DataFrame(X),pd.DataFrame(y),empty

## preprocess 

In [76]:
X_train,y_train,e=prepro(df_dir)

In [77]:
X_test,y_test,e=prepro(test_df_dir)

## i didn't do scale for the data because i am using trees 

## explore the data

In [78]:
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 246, 1: 54, 2: 69}

In [79]:
unique2, counts2 = np.unique(y_test, return_counts=True)
dict(zip(unique2, counts2))

{0: 38, 1: 35, 2: 40}

## using random forest 

In [157]:
params = {'min_impurity_decrease':[0, 0.1, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
              'max_features':['auto', 'sqrt', 'log2'],
              'n_estimators':list(range(1, 50))}

gsr = GridSearchCV(estimator=RandomForestClassifier(),cv=2,param_grid=params)

gsr.fit(X_train, y_train.values.ravel())

print("Best parameters set is:\n")
print(gsr.best_params_)

y_predr = gsr.predict(X_test)
print(accuracy_score(y_test, y_predr))

Best parameters set is:

{'max_features': 'auto', 'min_impurity_decrease': 0, 'n_estimators': 44}
0.46017699115044247


In [152]:
rf=RandomForestClassifier()

In [153]:
rf.fit(X_train, y_train.values.ravel())

RandomForestClassifier()

In [154]:
rf.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 2,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 2, 0, 0, 0, 2, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 2, 0,
       0, 2, 0], dtype=int64)

In [155]:
 accuracy_score(rf.predict(X_test), y_test)

0.45132743362831856

In [156]:
 accuracy_score(rf.predict(X_train), y_train)

0.997289972899729

## we can see a lot of overfitting :( let us try grid search on other models we will use DecisionTreeClassifier and LogisticRegression

In [145]:
tree_params = {'criterion':['gini','entropy'],'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}
gs = GridSearchCV(DecisionTreeClassifier(), tree_params, cv=2)
gs.fit(X_train, y_train)
print("Best parameters set is:\n")
print(gs.best_params_)

y_pred = gs.predict(X_test)
print(accuracy_score(y_test, y_pred))

Best parameters set is:

{'criterion': 'gini', 'max_depth': 5}
0.504424778761062


still not that good, hmmm

In [149]:
params =  {'penalty':['l1', 'l2', 'elasticnet', 'none'],'max_iter':[50,100,200,400,800]}

gs1 = GridSearchCV(LogisticRegression(), params, cv=2)
gs1.fit(X_train, y_train.values.ravel())

print("Best parameters set is:\n")
print(gs1.best_params_)

y_pred1 = gs1.predict(X_test)
print(accuracy_score(y_test, y_pred1))

C:\AI\envs\gpupy\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\AI\envs\gpupy\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_

C:\AI\envs\gpupy\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\AI\envs\gpupy\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_

Best parameters set is:

{'max_iter': 800, 'penalty': 'l2'}
0.6106194690265486


C:\AI\envs\gpupy\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


## so we can see that the first place goes to logistic Regression with Grid Search with the result:


Best parameters set is:

{'max_iter': 800, 'penalty': 'l2'}


0.6106194690265486